In [1]:
from functools import partial

import torch
import torch.nn.functional as F

In [2]:
torch.set_grad_enabled(False);

In [3]:
reduction = torch.mean

In [4]:
def ragged_gather(x, indices):
    # Is there a better torch func for this?
    # couldn't figger out how to get gather() to do this.
    return torch.tensor([x_i[y_i] for (x_i, y_i) in zip(x, y)])

In [5]:
def nll_loss_reimpl(x, y):
    return reduction(-ragged_gather(x, y))

In [6]:
N = 2  # batch
C = 3  # classes
probs = torch.tensor([
    [0.1, 0.2, 0.7],
    [0.3, 0.5, 0.2],
])
labels = torch.tensor([2, 1])
log_probs = torch.log(probs)
print(log_probs)
print(F.log_softmax(probs, dim=0))

tensor([[-2.3026, -1.6094, -0.3567],
        [-1.2040, -0.6931, -1.6094]])
tensor([[-0.7981, -0.8544, -0.4741],
        [-0.5981, -0.5544, -0.9741]])


In [7]:
x = log_probs
y = labels
print(F.nll_loss(x, y))
print(nll_loss_reimpl(x, y))

tensor(0.5249)
tensor(0.5249)


In [8]:
def cross_entropy_reimpl(x, y):
    x_c = ragged_gather(x, y)
    sum_x_j = torch.sum(torch.exp(x), axis=1)  # Along C
    terms = -torch.log(torch.exp(x_c) / sum_x_j)
    return reduction(terms)

In [9]:
print(F.cross_entropy(x, y))
print(cross_entropy_reimpl(x, y))

tensor(0.5249)
tensor(0.5249)


In [10]:
def kinda_fixed_point_thing(f, count, x):
    print(f)
    for _ in range(count):
        x_new = f(x)
        print(x_new - x)
        x = x_new

In [11]:
softmax = partial(F.softmax, dim=0)
log_softmax = partial(F.log_softmax, dim=0)

In [12]:
count = 3
kinda_fixed_point_thing(softmax, count, probs)
print()
kinda_fixed_point_thing(log_softmax, count, probs)

functools.partial(<function softmax at 0x7ff98fe7fd90>, dim=0)
tensor([[ 0.3502,  0.2256, -0.0775],
        [ 0.2498,  0.0744,  0.1775]])
tensor([[ 0.0249,  0.0373, -0.0615],
        [-0.0249, -0.0373,  0.0615]])
tensor([[ 0.0125,  0.0186, -0.0305],
        [-0.0125, -0.0186,  0.0305]])

functools.partial(<function log_softmax at 0x7ff98fe7fea0>, dim=0)
tensor([[-0.8981, -1.0544, -1.1741],
        [-0.8981, -1.0544, -1.1741]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
